In [12]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, uniform
import pandas as pd
import matplotlib.image as mpimg
from scipy.misc import derivative
from scipy.integrate import quad
from scipy.interpolate import interp1d
from scipy.stats import norm, uniform

### In this case we'll explore the case for multiple data set. We consider that the total distance is given by 
$$

d_{\rm tot} = \sum_{i} \omega_i d_{i} \left(X_{\rm obs,i}, X_{\rm sim,i} \right)
$$
### where $i$ runs over the number of multiple data set. 
### In this example we'll use SN Ia and BAO data to constrain CPL model.



In [13]:
def total_distance(sn_obs, sn_err, sn_sim,
                     bao_obs, bao_err, bao_sim,
                     weights=None):
    if weights is None:
        weights = {'hubble': 1.0, 'sn': 1.0, 'bao': 1.0}

    d_sn = sn_distance(sn_obs, sn_err, sn_sim)
    d_bao = bao_distance(bao_obs, bao_err, bao_sim)
    
    # Weighted combination
    total_distance = (weights['sn'] * d_sn +
                     weights['bao'] * d_bao)
    
    return total_distance


def sn_distance(observed, errors, simulated):
    """Distance metric for Supernovae data (distance modulus)"""
    residuals = (observed - simulated) / errors
    return np.sum(residuals**2)

def bao_distance(observed, errors, simulated):
    """Distance metric for BAO data (distance ratios)"""
    residuals = (observed - simulated) / errors
    return np.sum(residuals**2)

In [14]:
Omega_m0 = 0.3
Omega_L0 = 0.7
h = 0.71
H0 = h *100
zvals = np.linspace(0,3,100)

In [15]:
# cosmological distances 

def hubble_normalized_cpl(z, w0, wa):
    
    matter_term = Omega_m0 * (1 + z)**3
    
    de_term = Omega_L0 * (1+z)**(3*(1+w0+wa)) * np.exp(-3*wa*z/(1+z))
    return  H0 * np.sqrt(matter_term + de_term)

def H():
       return 100*h

def HUBz(z,w0,wa):
    return H()*hubble_normalized_cpl(z,w0,wa)

def inverse_hubble_normalized_cpl(z,w0,wa):

    return 1./hubble_normalized_cpl(z,w0,wa)

def hubble_normalized_a(a,w0,wa):
    return hubble_normalized_cpl(1./a-1,w0,wa)

def hubble_prime_normalized_a(a,w0,wa):
    return derivative(hubble_normalized_a(a,w0,wa), a, dx=1e-6)

def D_H():
    """
    Hubble distance in units of MPc (David Hogg arxiv: astro-ph/9905116v4)
    """
    return 2997.98/h

def comoving_distance_cpl(z,w0,wa):
    return D_H() *quad(inverse_hubble_normalized_cpl(z,w0,wa),0, z,epsabs=1e-6, epsrel=1e-6, limit=200)[0]

# angular diameter distance in units of MPc
def angular_diameter_distance_cpl(z,w0,wa):
    """
    Angular diameter distance as function of redshift z
    in units of MPc
    """
    d_c = comoving_distance_cpl(z,w0,wa)/(1.+z)
    return d_c

# luminosity distance in units of MPc
def luminosity_distance_cpl(z,w0,wa):
    d_c = comoving_distance_cpl(z,w0,wa)
    return (1 + z) * d_c

def build_luminosity_distance_interpolator(w0,wa,zmax=1.5,npoints=150):
    zgrid = np.linspace(1e-4, zmax, npoints)
    dLgrid = np.array([luminosity_distance_cpl(z,w0,wa) for z in zgrid])
    _dL_interp = interp1d(zgrid, dLgrid, kind='cubic', bounds_error=False, fill_value='extrapolate')
    _zmax_interp = zmax

def luminosity_distance_fast(z):
    if hasattr('_dL_interp') and z <= _zmax_interp:
        return _dL_interp(z)
    else:
        return luminosity_distance_cpl(z,w0,wa)

def distance_modulus(z,w0,wa):
    try:
        dL = luminosity_distance_fast(z,w0,wa)
        return 5 * np.log10(dL) + 25
    except Exception as e:
        print(f"[ERROR] distance_modulus failed at z = {z}: {e}")
        return 1e6  # Return large chi2 penalty



In [16]:
# DATA: 
data = pd.read_csv('/Users/alfonsozapata/Documents/SimpleMC/simplemc/data/binned_pantheon.txt', sep=r'\s+')
zcmb = data['zcmb']


arr_hub = np.loadtxt('/Users/alfonsozapata/Documents/SimpleMC/simplemc/data/Hz_all.dat')
z_obs= arr_hub[:,0]
hub_obs = arr_hub[:,1]
error_obs = arr_hub[:,2]

In [17]:

def mu_sim(w0,wa): 
    mod = np.array([distance_modulus(z,w0,wa) for z in zcmb])
    return mod

def Hubble_sim(w0, wa):
    try:
        Hz_sim = np.array([hubble_normalized_cpl(z, w0, wa) for z in z_obs])
        # Check for   # for NaN entries 
        if np.any(~np.isfinite(Hz_sim)) or np.any(Hz_sim <= 0):
            return None
        return Hz_sim
  
    except:
        return None

    

In [ ]:
# simulation of the data
#mu_sim = mu_sim(w0,wa)
#Hz_sim = Hubble_sim(w0,wa)
def dist1_sn(mu_obs,mu_sim,cov_filename):
    filename = cov_filename
        
    print("Loading covariance from {}".format(filename))
    f = open(filename)
    line = f.readline()
    n = int(len(self.zcmb))
    C = np.zeros((n,n))
    ii = -1
    jj = -1
    mine = 999
    maxe = -999
    for i in range(self.origlen):
        jj = -1
        if self.ww[i]:
            ii += 1
        for j in range(self.origlen):
            if self.ww[j]:
                jj += 1
            val = float(f.readline())
            if self.ww[i]:
                if self.ww[j]:
                    C[ii,jj] = val
    f.close()
    print('Done')
    self.cov = C
    self.xdiag = 1/self.cov.diagonal()



    data = pd.read_csv(values_filename, delim_whitespace=True)
    origlen = len(data)
    tvec = mu_obs - mu_sim
    tvec -= (tvec * xdiag).sum() / (xdiag.sum())
   '''
   The intrinsic supernova absolute magnitude M 
   is marginalized out by subtracting a weighted mean:
   '''
    
    return abs(np.mean(mu_obs) - np.mean(mu_sim))



def dist2_hub(observed, errors, simulated):
    if simulated is None:
        return np.inf
    
    return np.sum((observed - simulated)**2 / errors**2)


In [19]:
# we need define our priors for the parameters w0 and wa 
def prior_w0():
    return uniform.rvs(loc=-2.0, scale=1.0)
def prior_wa():
    return uniform.rvs(loc=-1.0, scale=2.0)
prior_samples_0 = [prior_w0() for _ in range(10)]
prior_samples_a = [prior_wa() for _ in range(10)]

In [23]:
def rejection_sim_cpl(prior_w0, prior_wa, zcmb, z_obs, observed_data_mu, observed_data_h, errors_mu, errors_h,
                     epsilon, n_accepted, n_sims_before_update=100000, max_simulations=10**7, verbose=True):
    """
    ABC rejection sampling for CPL dark energy parameters using both distance modulus and Hubble data
    
    Parameters:
    -----------
    prior_w0, prior_wa: functions that sample from priors for w0 and wa
    zcmb: array of CMB redshifts for distance modulus calculation using compressed data in this case
    z_obs: array of redshifts for Hubble parameter calculation  
    observed_data_mu: observed distance modulus data
    observed_data_h: observed Hubble parameter data
    errors_mu: errors for distance modulus data
    errors_h: errors for Hubble parameter data
    epsilon: tolerance threshold for acceptance
    n_accepted: target number of accepted samples
    n_sims_before_update: frequency of progress updates
    max_simulations: maximum number of simulations before stopping
    verbose: whether to print progress information
    """
    
    # Define the simulation functions
    def mu_sim(w0, wa): 
        """Simulate distance modulus for given w0, wa parameters"""
        mod = np.array([distance_modulus(z, w0, wa) for z in zcmb])
        return mod

    def Hubble_sim(w0, wa):
        """Simulate Hubble parameter for given w0, wa parameters"""
        try:
            Hz_sim = np.array([hubble_normalized_cpl(z, w0, wa) for z in z_obs])
            # Check for invalid values
            if np.any(~np.isfinite(Hz_sim)) or np.any(Hz_sim <= 0):
                return None
            return Hz_sim
        except:
            return None

    def combined_distance(observed_data_mu, errors_mu, observed_data_h, errors_h, simulated_data_mu, simulated_data_h):
        """
        Combined distance metric using both distance modulus and Hubble data
        Uses chi-squared like distance metric
        """
        if simulated_data_mu is None or simulated_data_h is None:
            return np.inf  # Reject if simulation failed
            
        # Calculate chi-squared for distance modulus
        chi2_mu = np.sum(((observed_data_mu - simulated_data_mu) / errors_mu)**2)
        
        # Calculate chi-squared for Hubble parameter  
        chi2_h = np.sum(((observed_data_h - simulated_data_h) / errors_h)**2)
        
        # Combined distance (could weight these differently if needed)
        total_distance = chi2_mu + chi2_h
        
        return total_distance

    accepted_particles = []
    total_simulations = 0
    distances = []
    
    if verbose:
        print(f"Starting ABC rejection with combined distance modulus and Hubble data.")
        print(f"Need to accept {n_accepted} particles.")
        print(f"Tolerance: $\\epsilon$ = {epsilon}")
        print(f"Maximum simulations: {max_simulations}")
        print(f"Number of distance points: {len(zcmb)}")
        print(f"Number of Hubble points: {len(z_obs)}")
    
    while len(accepted_particles) < n_accepted and total_simulations < max_simulations:
        # Sample from prior
        w0_par = prior_w0()
        wa_par = prior_wa()
        
        try:
            # Simulate both datasets
            simulated_mu = mu_sim(w0_par, wa_par)
            simulated_h = Hubble_sim(w0_par, wa_par)
            
            # Check if simulations were successful
            if simulated_mu is None or simulated_h is None:
                total_simulations += 1
                continue
                
            # Calculate combined distance
            d = combined_distance(observed_data_mu, errors_mu, observed_data_h, errors_h, 
                                 simulated_mu, simulated_h)
            distances.append(d)
            
            # Accept if within tolerance
            if d <= epsilon:
                accepted_particles.append((w0_par, wa_par, d, simulated_mu, simulated_h))
                
                if verbose and len(accepted_particles) % max(1, n_accepted//10) == 0:
                    print(f"Accepted {len(accepted_particles)}/{n_accepted} particles. "
                          f"Acceptance rate: {len(accepted_particles)/total_simulations:.6f}")
            
        except Exception as e:
            if verbose:
                print(f"Simulation failed for w0={w0_par:.3f}, wa={wa_par:.3f}: {e}")
            continue
        
        total_simulations += 1
        
        if verbose and total_simulations % n_sims_before_update == 0:
            current_rate = len(accepted_particles)/total_simulations if total_simulations > 0 else 0
            print(f"Completed {total_simulations} simulations. "
                  f"Accepted: {len(accepted_particles)}. "
                  f"Current acceptance rate: {current_rate:.6f}")
    
    if verbose:
        if total_simulations >= max_simulations:
            print(f"Stopped early! Reached maximum simulations ({max_simulations}).")
        final_rate = len(accepted_particles)/total_simulations if total_simulations > 0 else 0
        print(f"Completed! Total simulations: {total_simulations}. "
              f"Final acceptance rate: {final_rate:.6f}")

    # Extract results
    accepted_w0 = np.array([p[0] for p in accepted_particles])
    accepted_wa = np.array([p[1] for p in accepted_particles])
    accepted_d = np.array([p[2] for p in accepted_particles])
    accepted_mu = np.array([p[3] for p in accepted_particles])  # All accepted distance moduli
    accepted_h = np.array([p[4] for p in accepted_particles])   # All accepted Hubble values
    
    return {
        'w0': accepted_w0,
        'wa': accepted_wa, 
        'distances': accepted_d,
        'mu_simulations': accepted_mu,
        'h_simulations': accepted_h,
        'total_simulations': total_simulations,
        'all_distances': np.array(distances)
    }


In [ ]:
 # Run ABC with combined data
epsilon_combined = 100  # May need adjustment for combined distance
n_accepted_combined = 200


accepted_w0_comb, accepted_wa_comb, accepted_d_comb, total_sim_comb = (prior_w0, prior_wa, zcmb, z_obs, observed_data_mu, hub_obs, errors_mu, hub,
                     epsilon, n_accepted, n_sims_before_update=100000, max_simulations=10**7, verbose=True)

NameError: name 'simulator_combined' is not defined